In [1]:
%gui qt5

In [17]:
import numpy as np
import napari

In [72]:
from PartSeg.segmentation_mask.stack_gui_main import MainWindow
from PartSegImage import GenericImageReader
from PartSegCore import state_store
from PartSegCore.segmentation.noise_filtering import GaussNoiseFiltering, DimensionType
state_store.develop = True

In [47]:
def apply_sitk(fun, data, *args):
    return sitk.GetArrayFromImage(fun(sitk.GetImageFromArray(data), *args))


In [11]:
view = napari.Viewer()

In [3]:
window = MainWindow()
window.show()

In [4]:
image = GenericImageReader.read_image("../../test_data/test_nucleus.tif")

In [5]:
window.settings.image = image

In [6]:
import SimpleITK as sitk

In [83]:
gaussed = GaussNoiseFiltering.noise_filter(image.get_channel(0)[0], image.spacing, arguments={"dimension_type": DimensionType.Layer, "radius": 1})

In [90]:
data = (gaussed > 2000).astype(np.uint8)

In [85]:
seg = apply_sitk(sitk.MorphologicalWatershed, data, 0.51)
seg.max()

5179

In [33]:
seg = sitk.GetArrayFromImage(sitk.MorphologicalWatershed(sitk.GetImageFromArray(image.get_channel(0)[0]), 1))

In [ ]:
window.settings.set_segmentation(seg)

In [18]:
seg.shape, image.shape

((45, 509, 481), (1, 45, 509, 481, 1))

In [86]:
dist = apply_sitk(sitk.SignedMaurerDistanceMap, data)

In [43]:
view.add_labels(seg)

<Labels layer 'seg' at 0x7fb3fb752550>

In [74]:
view.add_image(image.get_channel(0), name="image")

<Image layer 'image' at 0x7fb3ee0550b8>

In [75]:
view.add_image(gaussed, name="gauss")

<Image layer 'gauss' at 0x7fb3c91222b0>

In [87]:
view.add_image(dist, blending="additive")

<Image layer 'dist' at 0x7fb3f8a8a358>

In [76]:
view.add_image(gaussed > 5000, blending="additive")

<Image layer 'Image' at 0x7fb3f8ca2da0>

In [88]:
seg = apply_sitk(sitk.MorphologicalWatershed, dist, 0.51)
seg.max()

6102

In [91]:
window.settings.set_segmentation(data)